## 1. Dependências

In [ ]:
import json
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, concatenate, Conv1D, MaxPooling1D, Flatten, Dropout

from sklearn.model_selection import train_test_split

## 2. Configurações e Utils

In [ ]:
def plot_data(temperatures, vibrations, fails, start, stop):

    plt.style.use('dark_background')
    plt.rcParams['figure.figsize'] = (8,6)
    plt.rcParams['lines.linewidth'] = 3
    plt.rcParams['font.size'] = 15

    plt.figure(figsize=(15, 12))

    plt.subplot(3, 1, 1)
    plt.plot(temperatures[start:stop], color="yellow", lw=5)
    plt.title('Temperatura')

    plt.subplot(3, 1, 2)
    plt.plot(vibrations[start:stop], color="blue", lw=5)
    plt.title('Vibração')

    plt.subplot(3, 1, 3)
    plt.plot(fails[start:stop], color="red", marker='o', lw=0)
    plt.title('Falhas')

    plt.show()

def look_back(df, samples):
  rX1, rX2, rY = [], [], []

  for i in range(len(df) - samples):

      rX1.append(df['temperature'].iloc[i:(i + samples)])
      rX2.append(df['vibration'].iloc[i:(i + samples )])
      rY.append(df['fail'].iloc[i:(i + samples)])

  rX1 = np.array(rX1).reshape(len(rX1), samples, 1)
  rX2 = np.array(rX2).reshape(len(rX2), samples, 1)

  rY = np.array(rY)[:,-1]

  return rX1, rX2, rY


## 3. Dataset

## 3.1 Carregamento (ATENÇÃO)

In [ ]:
!rm -rf synthetic_data.zip
!rm -rf synthetic_data.json
!rm -rf __MACOSX/

In [ ]:
!gdown 11fi4C4pN-7BZQKTRME8Yz8j-SkJf4rhJ

In [ ]:
%%capture
!unzip synthetic_data.zip

In [ ]:
df = pd.read_json("synthetic_data.json")
df

In [ ]:
t,v, f = df["temperature"], df["vibration"], df["fail"]
plot_data( t, v, f, 0, 800)

## 3.2. Normalização

In [ ]:
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df)

normalized_df = pd.DataFrame(normalized_data, columns=df.columns)

normalized_df

## 3.3. Amostra e Balanceamento (TAREFA 1)

O códo a segui tão possui nenhum erro e não está incompleto. O problema está dentro da funcão look_back que pega amostras de janelas de tempo e transforma e dados de treinamento.

Se você quiser observar o problema que a versão original dessa função causa, pule esta tarefa por hora e implemente os modelos. Depois retorne e corrija o problema lá na função na secção 2.

In [ ]:
back_log = 50

epochs = 30
batch_size = 32

X1, X2, Y = look_back(normalized_df, back_log)

X = np.column_stack((X1, X2))

print(X.shape)
print(Y.shape)

input_train, input_test, labels_train, labels_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

# 4. Modelos

## 4.1. Série Temporal (TAREFA 2)

Crie uma rede LSTM com 50 estados seguindo o input de entrada X.shape[1] e X.shape[2].

Utilize um dropout se achar necessário antes da última camada.

Ao final deve ser uma rede de classificação binária indicando a probabilidade de falha em uma única saída.

In [ ]:
RNN_Model = Sequential()

###### Crie aqui seu modelo


In [ ]:
learning_rate = 0.00001
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_function = tf.keras.losses.BinaryCrossentropy()

RNN_Model.compile(optimizer=opt, loss=loss_function)

In [ ]:
RNN_Model.summary()

In [ ]:
tf.keras.utils.plot_model(RNN_Model, show_shapes=True)

In [ ]:
log = RNN_Model.fit(x=input_train, y=labels_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

In [ ]:
losses = log.history["loss"]
val_losses = log.history["val_loss"]

data_epochs = np.arange(0, epochs, 1)
plt.plot(data_epochs, losses, color="yellow", lw=5, label = 'Training Loss')
plt.plot(data_epochs, val_losses, color="red", lw=5, label = 'Validation Loss')

plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend(loc='upper right')

In [ ]:
eval_loss = RNN_Model.evaluate(input_test, labels_test, batch_size=batch_size)
print(f"Accuracy: {1-eval_loss}")

## 4.2. Convolução (TAREFA 3)

Crie uma rede CNN com 1 camada convolucional de 1 DIMENSÃO, Kernel 3x3, com 64 unidade e input de entrada X.shape[1] e X.shape[2].

Coloque um maxpolling1d nao saída dessa camada com um kernel de 2x2.

Não esqueça de achar a matriz multimencional para um vetor 1D para servir de entrada camada a camada FC de classficação. É esperado que essa camada recebca um vetor de 50 pocições se sua rede estiver correta.

Utilize um dropout se achar necessário antes da última camada.

Ao final deve ser uma rede de classificação binária indicando a probabilidade de falha em uma única saída.

In [ ]:
CNN_Model = Sequential()

#### Crie aqui seu modelo


In [ ]:
learning_rate = 0.00001
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_function = tf.keras.losses.BinaryCrossentropy()

CNN_Model.compile(optimizer=opt, loss=loss_function)

In [ ]:
CNN_Model.summary()

In [ ]:
tf.keras.utils.plot_model(CNN_Model, show_shapes=True)

In [ ]:
log = CNN_Model.fit(input_train, labels_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

In [ ]:
losses = log.history["loss"]
val_losses = log.history["val_loss"]

data_epochs = np.arange(0, epochs, 1)
plt.plot(data_epochs, losses, color="yellow", lw=5, label = 'Training Loss')
plt.plot(data_epochs, val_losses, color="red", lw=5, label = 'Validation Loss')

plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend(loc='upper right')

In [ ]:
eval_loss = CNN_Model.evaluate(input_test, labels_test, batch_size=batch_size)
print(f"Accuracy: {1-eval_loss}")

# 5. Testes

## 5.1 Analise Comparativa

In [ ]:
window = 100

start = 0
stop =  50

s = random.randint(0, len(input_test)-1-back_log)
X_test, Y_test = input_test[s:s+window], labels_test[s:s+window]

print(X_test.shape)
print(Y_test.shape)

RNN_predictions = RNN_Model.predict(X_test, batch_size=batch_size)
CNN_predictions = CNN_Model.predict(X_test, batch_size=batch_size)

data_epochs = np.arange(start, stop, 1)

plt.plot(data_epochs, Y_test[start:stop], color="red", marker='o', lw=0, label = 'REAL')
plt.plot(data_epochs, RNN_predictions[start:stop], color="blue", marker='o', lw=0, label = 'RNN')
plt.plot(data_epochs, CNN_predictions[start:stop], color="yellow", marker='o', lw=0, label = 'CNN')

plt.xlabel('Janela de Tempo')
plt.ylabel('Probabilidade de Falha')
plt.legend(loc='upper right')